# Algo 

- Label dilated grid dual using primal information + outer values 
- Propagate primal label in dilated grid
- 

In [1]:
import sys
sys.path.append('../utils')
import fvdb.nn as fvnn
import torch 
import igl
from diffusion_tensor import *
import mesh_tools as mt
import matplotlib.pyplot as plt
from fvdb_utils import show_vdb_marching_cubes, grid_to_VDB, vdb_marching_cubes
from tqdm import tqdm
import pymeshlab as ml


### Utils

In [48]:
def create_dilated_grid(grid:fvdb.GridBatch):
    dilated_ijk = grid.ijk.jdata.clone()
    dilated_ijk = dilated_ijk[:, None] + torch.tensor(mt.mesh_grid(3, True), device=dilated_ijk.device)
    dilated_ijk = dilated_ijk.view(-1, 3).long()
    return fvdb.sparse_grid_from_ijk(dilated_ijk, origins=grid.origins, voxel_sizes=grid.voxel_sizes)


def label_dilated_crust(source_tensor: fvnn.VDBTensor, dilated_grid: fvdb.GridBatch):
    '''Label dilated grid based on the normal orientation: 0 (intersects surface), 1 or -1 (resp. strictly outside/inside)'''
    dilated_grid = create_dilated_grid(source_tensor.grid)
    dilated_crust_tensor = grid_to_VDB(dilated_grid, torch.ones, additional_feat=[1])
    dilated_centers = dilated_grid.grid_to_world(dilated_grid.ijk.float())
    neighbors = source_tensor.grid.neighbor_indexes(dilated_crust_tensor.grid.ijk, 1)
    
    # assign existing voxels to 0
    in_source_mask = neighbors.jdata[:, 1, 1, 1]!=-1
    
    # --> Decide sign
    flat_neighbors = neighbors.jdata.view(-1, 27)
    normals, vstars, _, _ = DiffusionTensor.get_feature_data(source_tensor.get_global().jdata)
    local_delta = (dilated_centers.jdata.unsqueeze(1)-vstars[flat_neighbors])
    local_sdf = (local_delta*normals[flat_neighbors]).sum(-1)
    local_sdf[flat_neighbors==-1] = 0
    local_sdf = local_sdf.sum(-1)/(flat_neighbors!=-1).sum(-1)

    dilated_crust_tensor.feature.jdata = local_sdf[:, None]
    dilated_crust_tensor.feature.jdata[in_source_mask] = 0
    
    return dilated_crust_tensor

def label_dual_grid(dilated_grid: fvdb.GridBatch, dilated_crust_tensor: fvnn.VDBTensor, source_tensor: fvnn.VDBTensor):
    '''label dual of dilated grid based on crust information. Disambiguate internal vertices with normal information'''
    dual_grid = dilated_grid.dual_grid()
    dual_centers = dual_grid.grid_to_world(dual_grid.ijk.float())
    dilated_centers = dilated_grid.grid_to_world(dilated_grid.ijk.float())
    

    new_feature = dual_grid.splat_trilinear(dilated_centers, dilated_crust_tensor.feature)
    dual_tensor = fvdb.nn.VDBTensor(dual_grid, new_feature)
    
    # label unlabeled (internal) voxels 
    is_unlabeled = (dual_tensor.feature.jdata==0).squeeze()
    offset = -1 + torch.tensor([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 1, 1]], device=is_unlabeled.device)

    primal_ijk = (dual_tensor.grid.ijk.jdata[is_unlabeled][:, None, :] + offset[None, :]).view(-1, 3)
    primal_index = source_tensor.grid.ijk_to_index(primal_ijk, 1).jdata.view(-1, 8)
    normals, vstars, _, _ = DiffusionTensor.get_feature_data(source_tensor.get_global().jdata)

    guess_sign = (normals[primal_index]*(dual_centers.jdata[is_unlabeled][:, None, :]-vstars[primal_index])).sum(-1).mean(-1)

    dual_tensor.feature.jdata[is_unlabeled] = guess_sign.unsqueeze(-1)
    dual_tensor.feature.jdata = torch.sign(dual_tensor.feature.jdata)
    return dual_tensor

def assign_dilated_vstars(source_tensor: fvnn.VDBTensor, dilated_grid: fvdb.GridBatch):
    '''create vstars in adjacent voxels'''
    dilated_grid = create_dilated_grid(source_tensor.grid)
    dilated_tensor = grid_to_VDB(dilated_grid, torch.ones, additional_feat=[source_tensor.jdata.shape[-1]])

    neighbors = source_tensor.grid.neighbor_indexes(dilated_tensor.grid.ijk, 1)
    
    # assign existing voxels to 0
    in_source_mask = neighbors.jdata[:, 1, 1, 1]!=-1
    
    # --> Decide sign
    flat_neighbors = neighbors.jdata.view(-1, 27)
    
    out_feat = source_tensor.jdata[flat_neighbors]
    out_feat[flat_neighbors==-1] = 0

    out_feat = out_feat.sum(-2)/(flat_neighbors!=-1).sum(-1, True)

    dilated_tensor.feature.jdata = out_feat
    dilated_tensor.feature.jdata[in_source_mask] = source_tensor.jdata[neighbors.jdata[:, 1, 1, 1][in_source_mask]]

    return dilated_tensor, in_source_mask


In [56]:

def dual_contouring(dual_tensor: fvnn.VDBTensor):
    ''' dual_tensor: sign on corner vertices
        based on dual edges (neighbors in the graph)'''
    dual_neighbors = dual_tensor.grid.neighbor_indexes(dual_tensor.grid.ijk, 1).jdata
    sign_data = dual_tensor.jdata.sign().squeeze()
    if 0 in sign_data:
        print('WARNING, WRONG SIGN DATA')

    dkx = dual_neighbors[:, 2, 1, 1]
    active_dkx = ((sign_data*sign_data[dkx])==-1)*(dkx!=-1)

    dky = dual_neighbors[:, 1, 2, 1]
    active_dky = ((sign_data*sign_data[dky])==-1)*(dky!=-1)

    dkz = dual_neighbors[:, 1, 1, 2]
    active_dkz = ((sign_data*sign_data[dkz])==-1)*(dkz!=-1)
    
    dkx_face_idx = torch.tensor([[0, -1, -1], [0, 0, -1], [0, 0, 0], [0, -1, 0]], device=dkx.device)
    dky_face_idx = torch.tensor([[-1, 0, 0], [0, 0, 0], [0, 0, -1], [-1, 0, -1]], device=dky.device)
    dkz_face_idx = torch.tensor([[-1, -1, 0], [0, -1, 0], [0, 0, 0], [-1, 0, 0]], device=dkz.device)


    faces = []
    for dk_face_idx, active_dk in zip([dkx_face_idx, dky_face_idx, dkz_face_idx], [active_dkx,  active_dky, active_dkz]):
        face_idx = (dk_face_idx[:, None] + dual_tensor.grid.ijk.jdata[active_dk][None, :]).permute((1, 0, 2))
        face_idx[sign_data[active_dk]>0] = face_idx[sign_data[active_dk]>0].flip(1)
        faces.append(face_idx)
    return torch.concatenate(faces)

def faces_ijk_to_index(candidate_faces: torch.tensor, source_grid: fvdb.GridBatch):
    df = source_grid.ijk_to_index(candidate_faces.view(-1, 3)).jdata.view(-1, 4)
    return df

### Dev mode

In [59]:
ind=2
tens = torch.load('../../output/acropolis/gen_0_4.pt', weights_only=False)
source_tensor = DiffusionTensor(tens.grid[ind], tens.feature[ind]).remove_mask()

In [60]:


dilated_grid = create_dilated_grid(source_tensor.grid)
dilated_crust_tensor = label_dilated_crust(source_tensor, dilated_grid)
dual_tensor = label_dual_grid(dilated_grid, dilated_crust_tensor, source_tensor)
dilated_tensor, in_source_mask = assign_dilated_vstars(source_tensor.get_global(), dilated_grid)


candidate_faces = dual_contouring(dual_tensor)
faces = faces_ijk_to_index(candidate_faces, dilated_tensor.grid)
faces = faces[(faces!=-1).all(-1)]
_, vstars, _, _ = DiffusionTensor.get_feature_data(dilated_tensor.jdata)

In [72]:
vstars.cpu().detach().numpy()

array([[-0.88288313, -0.24309044, -0.40174899],
       [-0.88298166, -0.24316064, -0.39824712],
       [-0.88309115, -0.2430131 , -0.39441347],
       ...,
       [ 0.8307246 ,  0.19292234,  0.368812  ],
       [ 0.841833  ,  0.18112609,  0.36912334],
       [ 0.8451569 ,  0.18033078,  0.36972064]], dtype=float32)

In [79]:
faces.cpu().detach().numpy().max()

346728

In [77]:
vstars.cpu().double().detach().numpy().dtype

dtype('float64')

In [87]:
ms = ml.MeshSet()
nmesh = ml.Mesh(vstars.cpu().double().detach().numpy(), faces.cpu().detach().numpy().astype(np.int32))
ms.add_mesh(nmesh)
ms.save_current_mesh('test_pc.ply')

In [70]:
ml.Mesh?

Docstring:      <no docstring>
Init docstring:
__init__(*args, **kwargs)
Overloaded function.

1. __init__(self: pymeshlab.pmeshlab.Mesh, vertex_matrix: numpy.ndarray[numpy.float64[m, 3]] = array([], shape=(0, 3), dtype=float64), face_matrix: numpy.ndarray[numpy.int32[m, 3]] = array([], shape=(0, 3), dtype=int32), edge_matrix: numpy.ndarray[numpy.int32[m, 2]] = array([], shape=(0, 2), dtype=int32), v_normals_matrix: numpy.ndarray[numpy.float64[m, 3]] = array([], shape=(0, 3), dtype=float64), f_normals_matrix: numpy.ndarray[numpy.float64[m, 3]] = array([], shape=(0, 3), dtype=float64), v_scalar_array: numpy.ndarray[numpy.float64[m, 1]] = array([], dtype=float64), f_scalar_array: numpy.ndarray[numpy.float64[m, 1]] = array([], dtype=float64), v_color_matrix: numpy.ndarray[numpy.float64[m, 4]] = array([], shape=(0, 4), dtype=float64), f_color_matrix: numpy.ndarray[numpy.float64[m, 4]] = array([], shape=(0, 4), dtype=float64), v_tex_coords_matrix: numpy.ndarray[numpy.float64[m, 2]] = array(

In [68]:
tm = trimesh.Trimesh(vstars.cpu().detach().numpy(), faces.cpu().detach().numpy(), process=False)


In [ ]:
tm.save

In [61]:
mt.plotSlice(source_tensor.to_dense()[0, ..., 6].cpu().detach().numpy(), 1.)

interactive(children=(IntSlider(value=114, description='s', max=227), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [62]:
mt.plotSlice(dilated_crust_tensor.to_dense()[0, ..., 0].cpu().detach().numpy(), .001)

interactive(children=(IntSlider(value=115, description='s', max=229), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [63]:
# mt.export_obj(*vdb_marching_cubes(dual_tensor), 'mc4')

In [64]:
in_source_mask.shape

torch.Size([346783])

In [41]:
mp = plot(vstars.cpu().detach().numpy(), faces, shading={'wireframe':True})
mp.add_points(vstars[in_source_mask].cpu().detach().numpy(), shading={'point_size':.01})
mp.add_points
mt.export_obj(vstars.cpu().detach().numpy(), faces, 'dc4')


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000225…

### MC

In [89]:
ind=2
tens = torch.load('../../output/acropolis/gen_0_4.pt', weights_only=False)
source_tensor = DiffusionTensor(tens.grid[ind], tens.feature[ind]).remove_mask()

In [90]:
dilated_grid = create_dilated_grid(source_tensor.grid)
dilated_tensor, in_source_mask = assign_dilated_vstars(source_tensor.get_global(), dilated_grid)
dual_grid = dilated_tensor.grid.dual_grid()
dual_centers = dual_grid.grid_to_world(dual_grid.ijk.float())
dilated_centers = dilated_grid.grid_to_world(dilated_grid.ijk.float())
dual_tensor = grid_to_VDB(dual_grid, additional_feat=[1])
offset = -1 + torch.tensor([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 1, 1]], device=dual_grid.device)
primal_ijk = (dual_tensor.grid.ijk.jdata[:, None, :] + offset[None, :]).view(-1, 3)
primal_index = dilated_grid.ijk_to_index(primal_ijk, 1).jdata.view(-1, 8)
normals, vstars, _, _ = DiffusionTensor.get_feature_data(dilated_tensor.jdata)

guess_sign = (normals[primal_index]*(dual_centers.jdata[:, None, :]-vstars[primal_index])).sum(-1)
guess_sign[primal_index==-1] = 0
dual_tensor.feature.jdata = guess_sign.sum(-1, True)/(primal_index!=-1).sum(-1, True)
v, f = vdb_marching_cubes(dual_tensor)
ms = ml.MeshSet()
nmesh = ml.Mesh(v, f)
ms.add_mesh(nmesh)
ms.save_current_mesh('test_pc.ply')

In [93]:
mt.plotSlice(dual_tensor.to_dense()[0, ..., 0].cpu().detach().numpy(), .1)

interactive(children=(IntSlider(value=115, description='s', max=230), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [98]:
-dual_tensor.feature

TypeError: bad operand type for unary -: 'VDBTensor'